## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [ ]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [7]:
import requests
import pandas as pd
import json
from datetime import datetime
import regex as re

## Download

Retrieve the page

In [8]:
url = "https://services2.arcgis.com/SCn6czzcqKAFwdGU/arcgis/rest/services/COVID19Surveypt2v3_view2/FeatureServer/0//query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=City%2C+C_total_cases%2C+EditDate&orderByFields=Date_reported%20desc&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="

In [9]:
r = requests.get(url)

In [10]:
data = r.json()

## Parse

In [11]:
dict_list = []

In [12]:
for item in data["features"]:
    timestamp = item["attributes"]["EditDate"]
    timestamp = datetime.fromtimestamp((timestamp / 1000))
    d = dict(
        county="Solano",
        area=item["attributes"]["City"],
        confirmed_cases=item["attributes"]["C_total_cases"],
        county_date=timestamp,
    )
    dict_list.append(d)

In [13]:
df = pd.DataFrame(dict_list)

Parse the date column

In [14]:
df["county_date"] = df.county_date

In [15]:
df["confirmed_cases"] = df.confirmed_cases.fillna(0)

In [16]:
df["county_date"] = pd.to_datetime(df["county_date"]).dt.date

Get the latest data

In [17]:
latest_df = df[df.county_date == df.county_date.max()]

Clean up  city names for export

In [18]:
export_df = latest_df.replace("_", " ", regex=True)

In [19]:
export_df["confirmed_cases"] = export_df["confirmed_cases"].astype(int)

In [20]:
export_df = export_df.dropna(axis=0, subset=["area"])

In [21]:
export_df = export_df.drop_duplicates(subset=["county_date", "area"], keep="first")

## Vet

In [23]:
try:
    assert not len(export_df) > 9
except AssertionError:
    raise AssertionError(
        f"Solano County's scraper has extra {len(export_df) - 9} rows: {list(export_df.area)}"
    )

In [24]:
try:
    assert not len(export_df) < 9
except AssertionError:
    raise AssertionError(
        f"Solano County's scraper is missing {9 - len(export_df)} rows: {list(export_df.area)}"
    )

## Export

Set date

In [20]:
tz = pytz.timezone("America/Los_Angeles")

In [21]:
today = datetime.now(tz).date()

In [22]:
slug = "solano"

In [23]:
export_df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [24]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [25]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [26]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [27]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)